In [74]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds


In [75]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

def normalize_img(image, label):
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.cache()
ds_train = ds_train.batch(128)
ds_train = ds_train.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_test = ds_test.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [76]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(62, activation='softmax')
])
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)

model.fit(
    ds_train,
    epochs=5,
    validation_data=ds_test,
)

Epoch 1/5
469/469 [==============================] - 4s 9ms/step - loss: 0.4443 - accuracy: 0.8831 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/5
469/469 [==============================] - 2s 5ms/step - loss: 0.1952 - accuracy: 0.9461 - val_loss: 0.1540 - val_accuracy: 0.9552
Epoch 3/5
469/469 [==============================] - 2s 5ms/step - loss: 0.1422 - accuracy: 0.9597 - val_loss: 0.1270 - val_accuracy: 0.9623
Epoch 4/5
469/469 [==============================] - 2s 5ms/step - loss: 0.1117 - accuracy: 0.9683 - val_loss: 0.1113 - val_accuracy: 0.9663
Epoch 5/5
469/469 [==============================] - 2s 4ms/step - loss: 0.0911 - accuracy: 0.9749 - val_loss: 0.1004 - val_accuracy: 0.9689


In [79]:
model.save('MNIST_.31%_Carlo_Pastoral.h5')

In [78]:
test_loss, test_acc = model.evaluate(ds_test, verbose=2)
print('\nTest accuracy:', test_acc)

image, label = tfds.as_numpy(tfds.load(
    'mnist',
    split='test', 
    batch_size=-1, 
    as_supervised=True,
))

predictions = model.predict(image) 
print(np.argmax(predictions[50:60],axis=1))
print(label[50:60])

79/79 - 0s - loss: 0.1004 - accuracy: 0.9689

Test accuracy: 0.9689
[0 7 9 9 9 6 5 8 8 6]
[0 7 9 9 9 6 5 8 8 6]
